In [1]:
import timm
import torch
import torch.nn as nn
import sys
import os
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision import datasets, transforms
sys.path.append(os.path.abspath(".."))
from data.ImageDataset import ImageDataset

In [2]:
model = timm.create_model('xception', pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),                             
    nn.Dropout(p=0.5),                     
    nn.Linear(512, 1),                     
    nn.Sigmoid()                          
)



/home/ec2-user/CS230Project/venv/lib64/python3.9/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(


In [3]:
criterion = nn.BCELoss()
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [4]:
transform = transforms.Compose([
    transforms.Resize((299, 299)),           
    transforms.ToTensor(),                    
    transforms.Normalize(                     
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    ),
])

train_dataset = ImageDataset(
    annotations_path="/home/ec2-user/CS230Project/data/annotations/train.json",
    images_dir="/home/ec2-user/CS230Project/data/train",
    transform=transform,
)

val_dataset = ImageDataset(
    annotations_path="/home/ec2-user/CS230Project/data/annotations/val.json",
    images_dir="/home/ec2-user/CS230Project/data/val",
    transform=transform,
)

train_loader = DataLoader(train_dataset, batch_size=16, num_workers=7,shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=7, shuffle=False)

In [ ]:
num_epochs = 10
best_val_acc = float("-inf")
for epoch in range(num_epochs):

    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device).float()  


        outputs = model(images)
        outputs = outputs.view(-1)  
        labels = labels.view(-1)  
        loss = criterion(outputs, labels)

    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted = (outputs > 0.5).float()  
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%")

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device).float()

            outputs = model(images)
            outputs = outputs.view(-1)  
            labels = labels.view(-1)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = 100. * correct / total
    print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Accuracy: {val_accuracy:.2f}%")

    scheduler.step()
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        checkpoint_path = f"/home/ec2-user/CS230Project/code/models/saved-weights/ExceptionNet/exception_net_{epoch+1}.pth"
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Model saved to {checkpoint_path}")



Training Epoch 1/10: 100%|██████████| 2757/2757 [13:32<00:00,  3.39it/s]


Epoch 1, Train Loss: 0.4618, Accuracy: 77.31%


Validation: 100%|██████████| 457/457 [00:54<00:00,  8.40it/s]


Validation Loss: 0.3222, Accuracy: 86.73%
Model saved to /home/ec2-user/CS230Project/code/models/saved-weights/ExceptionNet/exception_net_1.pth


Training Epoch 2/10: 100%|██████████| 2757/2757 [13:18<00:00,  3.45it/s]


...

Training Epoch 10/10: 100%|██████████| 2757/2757 [13:10<00:00,  3.49it/s]


Epoch 10, Train Loss: 0.0217, Accuracy: 98.78%


Validation: 100%|██████████| 457/457 [00:45<00:00, 10.15it/s]

Validation Loss: 1.0517, Accuracy: 88.82%


: 